In [ ]:
# I would assume you need to import VIVA data from yfinance to then call the Financial_Rations_Metrics.py 
# functions to calculate the metrics and table them?

# Note this is no where near complete this is just an example.

import yfinance as yf

# Our ticker Viva Energy
TICKER = "VEA.AX"

try:
    # Get stock information
    stock = yf.Ticker(TICKER)
    stock_prices = stock.history(period="10y")
    stock_info = stock.info
    stock_financials = stock.financials
    stock_balance_sheet = stock.balance_sheet

    # Extract relevant information
    last_price = stock_prices['Close'].iloc[-1]
    earnings = stock_financials.loc["Net Income"].iloc[0]
    dividend = stock_info['lastDividendValue']
    shares_outstanding = stock_info['sharesOutstanding']
    total_assets = (stock_balance_sheet.loc["Total Assets"].iloc[0] + stock_balance_sheet.loc["Total Assets"].iloc[1]) / 2

    print(f"Chosen Stock: {TICKER} \nLast Share Price: {last_price:.2f} \nTotal Assets: {total_assets}")
      
except Exception as e:
    print(f"An error occurred: {e}")



Chosen Stock: VEA.AX 
Last Share Price: 1.65 
Total Assets: 10603800000.0
